GLF Kernel - Tutorial
===

#### Installation
See the instructions in the GitHub repository ([https://github.com/kwarc/glf_kernel](https://github.com/kwarc/glf_kernel)).

#### Very Brief Intro: Jupyter Cells
In Jupyter, there are two types of cells:
* *markdown cells* (such as this one), which can be used for documentation and explanations
* *code cells*, which contain GF grammars and shell commands if you use the GF kernel

You can edit a cell by clicking on it (double clicking in the case of markdown cells).
To execute the cell, you can press <kbd>Ctrl</kbd>+<kbd>Return</kbd> or <kbd>Shift</kbd>+<kbd>Return</kbd>. If you want to execute all cells in the notebook, you can choose *Cell* -> *Run All* in the top menu.

## GLF Kernel Commands

You can use this kernel to create new MMT archives or switch to existing ones using the `archive` command.

`archive -c 'archive-name'` will create a new archive.

`archive 'archive-name'` will set your current working archive to the speciified archive.

The default archive for this kernel will be `COMMA/JUPYTER`

In [1]:
archive COMMA/JUPYTER

Changed to archive COMMA/JUPYTER

## Defining Grammars
Now you can define GF Grammars directly in the notebook using code cells.

These grammars will automatically be saved into the source directory of the specified archive.

For toggling line numbers in a cell, you need to switch to command mode (<kbd>Esc</kbd>) and press <kbd>L</kbd>.

In [2]:
abstract Grammar = {
    cat
        Person ; Action ; Sentence ;

    fun
        john, mary : Person ;
        run, be_happy : Action ;
        make_sentence : Person -> Action -> Sentence ;
        and : Sentence -> Sentence -> Sentence ;
}

Defined Grammar

In [3]:
concrete GrammarEng of Grammar = {
    lincat
        Person = Str ;
        Action = Str ;
        Sentence = Str ;
    lin
        john = "John" ;
        mary = "Mary" ;
        run = "runs" ;
        be_happy = "is happy" ;
        make_sentence person action = person ++ action ;
        and a b = a ++ "and" ++ b ;
}

If you also want to have one of your grammars as a `.gf` file, you can export it with the kernel command `export`.

In [21]:
export Grammar

Exported Grammar.gf

You can also run normal GF commands, as you are used to

In [5]:
parse -cat=Sentence "John runs"

make_sentence john run

For displaying graphs in the notebook, you need to pipe them into the `show` command

In [6]:
parse -cat=Sentence "John runs and Mary is happy" | view_tree | show

Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x03\x05\x00\x00\x00\xfb\x08\x06\x00\x00\x00;\x158\x8…

## MMT For Target Logic and Semantics Construction

You can also use it to define new theories and views that you would like to use with your grammars.
Every time a new theory, view or grammar is specified MMT will build your current archive.

The namespace of defined theories or views will automatically be set to the current archive.

In [8]:
theory FolLogic : ur:?LF =
	o : type ❙
	ι : type ❙
	and : o ⟶ o ⟶ o ❘ # 1 ∧ 2 ❙
❚

Created theory FolLogic

In [9]:
theory DomainTheory : ?FolLogic =
	john : ι ❙
	mary : ι ❙
	run : ι ⟶ o ❙
	happy : ι ⟶ o ❙
❚

Created theory DomainTheory

In [10]:
view grammarSem : JUPYTER/Grammar.gf?Grammar -> ?DomainTheory =
	Person = ι ❙
	Action = ι ⟶ o ❙
	Sentence = o ❙
	john = john ❙
	mary = mary ❙
	run = run ❙
	be_happy = happy ❙
	make_sentence = [p,a] (a p) ❙
	and = [a,b] b ∧ a ❙
❚

Created view grammarSem

Now you can send requests to the GLF-Server using the `request` command.
By default `request` uses the last view you defined and a takes list of abstract syntax trees separated by vertical bars as argument.

In [14]:
parse -cat=Sentence "John runs and Mary is happy" | construct -v grammarSem

(happy mary)∧(run john)

Should you wish to change the current view that is used for the requests you can do it via the -v option.
This also sets the specified view as the default view for further requests.

## Further Kernel Commands

Apart from `export` and `view`, there are two more kernel commands: `clean` and `help`.

`clean` removes `.gfo`, `.dot` and `.png` files.

`help` displays information on the kernel commands (not the GF shell commands).
If you want information on the GF shell commands instead, you should use
the shortcut `h` instead:

In [12]:
h reload

r, reload

repeat the latest import command

In [19]:
help

Available kernel commands:

show 'graph' : shows the graph in the notebook

clean : remove all .dot, .png and .gfo files from the current directory

export 'name' : export the grammar 'name' to your current directory

archive [-c] 'MMT-archive' : sets 'MMT-archive' as the current archive. The -c option creates the archive if it does not exist yet

...

To display more information on the GF shell commands use 'h'

All kernel commands are pipeable in a GF-shell like manner using '|', e.g. parse 'Sentence' | vt | view

You can use the kernel as an editor for your grammars, views or theories.

Stated grammars are automatically imported upon definiton.

Whenever a grammar, theory or view is defined the current MMT archive will be built